In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1.]
[1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-15 10:58:56,036	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-15_10-58-56_2958/logs.
2019-06-15 10:58:56,147	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:31522 to respond...
2019-06-15 10:58:56,283	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:49754 to respond...
2019-06-15 10:58:56,289	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-15 10:58:56,333	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-15_10-58-56_2958/logs.
2019-06-15 10:58:56,335	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.105',
 'redis_address': '192.168.3.105:31522',
 'object_store_address': '/tmp/ray/session_2019-06-15_10-58-56_2958/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-15_10-58-56_2958/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks
import games_configurations

a2c_config = games_configurations.bipedalwalker_config
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
agent = A2CAgent(sess,'walker', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


2019-06-15 10:58:57,448	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


Instructions for updating:
Use tf.cast instead.
(pid=3061) 
(pid=3061) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=3061) For more information, please see:
(pid=3061)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=3061)   * https://github.com/tensorflow/addons
(pid=3061) If you depend on functionality not listed there, please file an issue.
(pid=3061) 
(pid=3059) 
(pid=3059) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=3059) For more information, please see:
(pid=3059)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=3059)   * https://github.com/tensorflow/addons
(pid=3059) If you depend on functionality not listed there, please file an issue.
(pid=3059) 
(pid=3064) 
(pid=3064) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=3064) For more information, please see:
(pid=3064)   * https:

(pid=3105) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=3105)   result = entry_point.load(False)
(pid=3105) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=3105) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=3090) 
(pid=3090) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=3090) For more information, please see:
(pid=3090)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=3090)   * https://github.com/tensorflow/addons
(pid=3090) If you depend on functionality not listed there, please file an issue.
(pid=3090) 
(pid=3063) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=3063) WARN: gym.space

Frames per seconds:  2383.05884494006
Frames per seconds:  2301.747021070524
Frames per seconds:  2216.5345165950416
saving next best rewards:  -40.21512586042345
Frames per seconds:  2172.477458035332
Frames per seconds:  2439.0529110985417
saving next best rewards:  -27.18632268862389
Frames per seconds:  1947.6293648968083
saving next best rewards:  -20.871300624779657
Frames per seconds:  2377.933657360404
Frames per seconds:  2035.943467178872
Frames per seconds:  2171.3791325084726
saving next best rewards:  -19.56130346278071
Frames per seconds:  2196.0304700908728
Frames per seconds:  2196.931636030874
saving next best rewards:  -9.137995992908014
Frames per seconds:  2033.028224989539
saving next best rewards:  0.8275104569058033
Frames per seconds:  2467.678151990884
Frames per seconds:  2425.9472116645175
Frames per seconds:  2449.711612562753
saving next best rewards:  1.9347315061967247
Frames per seconds:  2544.4966966464376
Frames per seconds:  2649.920908050592
saving n

In [ ]:
ray.shutdown()

In [ ]:
import numpy as np
#agent.save('nn/latest')
#agent.restore('nn/carRoboschoolAnt-v1')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
env = env_configurations.configurations[env_name]['ENV_CREATOR']()
def evaluate(env,t_max=5000):
    rewards = []
    #env._max_episode_steps = 5000
    env.batch_mode = True
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env.close()




In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()